In [109]:
import numpy as np
import os
#from scipy.misc import imresize
import imageio
from PIL import Image
from skimage.transform import resize
import datetime
import os
#from google.colab import drive
import warnings
#ignore('warnings')
warnings.filterwarnings('ignore')

In [110]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
#tf.set_random_seed(30)
tf.random.set_seed(30)

In [111]:
#drive.mount('/content/gdrive/')

In [112]:
#!unzip /content/gdrive/My\ Drive/DeepLearning/GestureRecog_CaseStudy/Project_data.zip
#!ls
#!pwd

In [113]:

#train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
#val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
#batch_size = #experiment with the batch size

train_doc = np.random.permutation(open('C:\\Users\\rajansa\\OneDrive - Lam Research\\QA Folder\\_myLearning\\DeepLearning\\Gesture_Recognition\\Project_data\\train.csv').readlines())
val_doc = np.random.permutation(open('C:\\Users\\rajansa\\OneDrive - Lam Research\\QA Folder\\_myLearning\\DeepLearning\\Gesture_Recognition\\Project_data\\val.csv').readlines())
batch_size = 64 #experiment with the batch size

In [114]:
len(val_doc)

100

In [115]:
def generator1(source_path, folder_list, batch_size):
  #print( 'Source path = ', source_path, '; batch size =', batch_size)
  print( 'Source path = ', source_path, '; batch size =', batch_size)
  img_idx = 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17  #create a list of image numbers you want to use for a particular video
  #img_idx1 = 0,1,2
  x = len(img_idx)
  print("Length of Source Path",len(source_path))
  print("Length of batch Size",batch_size)
  while True:
    t = np.random.permutation(folder_list)
    num_batches = len(source_path)//batch_size # calculate the number of batches
    for batch in range(num_batches): # we iterate over the number of batches
      batch_data = np.zeros((batch_size,18,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
      batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
      for folder in range(batch_size): # iterate over the batch_size
        imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
        for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
          image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

          #crop the images and resize them. Note that the images are of 2 different shape
          #and the conv3D will throw error if the inputs in a batch have different shapes

          
          #h, w, c = image.shape
          #print(image.shape)
          #image = resize(image, (height, width), anti_aliasing=True)
          if image.shape[1] == 160:
            image = resize(image[:,20:140,:],(100,100)).astype(np.float32)
          else:
            #print("")
            image=resize(image,(100,100)).astype(np.float32)

          batch_data[folder,idx,:,:,0] = image[:,:,0] - 104 #normalise and feed in the image
          batch_data[folder,idx,:,:,1] = image[:,:,1] - 117  #normalise and feed in the image
          batch_data[folder,idx,:,:,2] = image[:,:,2] - 123  #normalise and feed in the image

          batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
      yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do



    # write the code for the remaining data points which are left after full batches
    #print(len(t))
    if(len(t)%batch_size!=0):
      #new_batch_size = len(folder_list) - (batch_size*num_batches)
      #print("New Batch Size", new_batch_size)
      batch_data = np.zeros((len(t)%batch_size,18,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
      #print("New Batch Size", len(t)%batch_size)
      batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
      #print("New Batch Size", new_batch_size)
      #print("New Batch Size", type(new_batch_size))
      for folder in range(len(t)%batch_size): # iterate over the batch_size
        #print("folder Batch Size", folder)
        #print("folder Batch Size", type(folder))
        imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*len(t)%batch_size)].split(';')[0]) # read all the images in the folder
        for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
          image = imageio.imread(source_path+'/'+ t[folder + (num_batches*len(t)%batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

          #crop the images and resize them. Note that the images are of 2 different shape
          #and the conv3D will throw error if the inputs in a batch have different shapes

          if image.shape[1] == 160:
            image = resize(image[:,20:140,:],(100,100)).astype(np.float32)
          else:
            image = resize(image,(100,100)).astype(np.float32)
          # Normalizing RGB image with mean subtraction
          batch_data[folder,idx,:,:,0] = image[:,:,0] - 104   #normalise and feed in the image
          batch_data[folder,idx,:,:,1] = image[:,:,1] - 117   #normalise and feed in the image
          batch_data[folder,idx,:,:,2] = image[:,:,2] - 123   #normalise and feed in the image

          #batch_data[folder,idx,:,:,0] = batch_data[folder,idx,:,:,0]/255 #normalise and feed in the image
          #batch_data[folder,idx,:,:,1] = batch_data[folder,idx,:,:,1]/255  #normalise and feed in the image
          #batch_data[folder,idx,:,:,2] = batch_data[folder,idx,:,:,2]/255  #normalise and feed in the image

          batch_labels[folder, int(t[folder + (num_batches*len(t)%batch_size)].strip().split(';')[2])] = 1
      yield batch_data, batch_labels


In [116]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:\\Users\\rajansa\\OneDrive - Lam Research\\QA Folder\\_myLearning\\DeepLearning\\Gesture_Recognition\\Project_data\\train'
val_path = 'C:\\Users\\rajansa\\OneDrive - Lam Research\\QA Folder\\_myLearning\\DeepLearning\\Gesture_Recognition\\Project_data\\val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [117]:
train_generator = generator1(train_path, train_doc, batch_size)
#print(train_generator)

## Model

Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [118]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model = Sequential()
model.add(Conv3D(64,(3,3,3),padding='same',input_shape=(18,100, 100,3)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(128,(3,3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
#model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2)))

model.add(Conv3D(256,(3,3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2, 2,2), strides=(2,2,2)))

model.add(Conv3D(256,(3,3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2, 2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation = 'elu'))
#model.add(Activation('elu'))
model.add(Dropout(0.5))
#model.add(Dense(len(class_names)))
model.add(Dense(5,activation = 'softmax'))


In [119]:
optimiser = optimizers.SGD(learning_rate=0.001,momentum=0.9,nesterov=True) 
 #,weight_decay=None,clipnorm=None,clipvalue=None,global_clipnorm=None,use_ema=False,ema_momentum=0.99,ema_overwrite_frequency=None,jit_compile=True,) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_28 (Conv3D)          (None, 18, 100, 100, 64)  5248      
                                                                 
 batch_normalization_28 (Bat  (None, 18, 100, 100, 64)  256      
 chNormalization)                                                
                                                                 
 activation_28 (Activation)  (None, 18, 100, 100, 64)  0         
                                                                 
 max_pooling3d_28 (MaxPoolin  (None, 9, 50, 100, 64)   0         
 g3D)                                                            
                                                                 
 conv3d_29 (Conv3D)          (None, 9, 50, 100, 128)   221312    
                                                                 
 batch_normalization_29 (Bat  (None, 9, 50, 100, 128) 

In [120]:
batch_size = 64
train_generator = generator1(train_path, train_doc, batch_size)
val_generator = generator1(val_path, val_doc, batch_size)

In [121]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
#filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience =2, verbose=1, mode='min',min_delta=0.0001, cooldown=0, min_lr = 0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [122]:
print(type(model_name))

<class 'str'>


In [123]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [124]:
#model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                    callbacks=callbacks_list, validation_data=val_generator, 
#                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, 
                    class_weight=None, workers=1, initial_epoch=0)

Source path =  C:\Users\rajansa\OneDrive - Lam Research\QA Folder\_myLearning\DeepLearning\Gesture_Recognition\Project_data\train ; batch size = 64
Length of Source Path 114
Length of batch Size 64
Epoch 1/30
11/11 [==============================] - ETA: 0s - loss: 4.8779 - categorical_accuracy: 0.3327 Source path =  C:\Users\rajansa\OneDrive - Lam Research\QA Folder\_myLearning\DeepLearning\Gesture_Recognition\Project_data\val ; batch size = 64
Length of Source Path 112
Length of batch Size 64

Epoch 1: saving model to model_init_2023-08-3115_02_13.740031\model-00001-4.87794-0.33267-17.33815-0.19000.h5
11/11 [==============================] - 809s 72s/step - loss: 4.8779 - categorical_accuracy: 0.3327 - val_loss: 17.3381 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 2/30
 5/11 [============>.................] - ETA: 5:56 - loss: 2.6107 - categorical_accuracy: 0.4112

KeyboardInterrupt: 